In [0]:
storageAccount="cookbookadlsgen2storage1"
mountpoint = "/mnt/Gen2Source"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID =""
tenantID =""
clientSecret =""
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except e:
  print("Already Mounted")
    


Mount Point =/mnt/Gen2Source

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,sum,avg,max

In [0]:
df_parquet = spark.read.format("parquet").option("header",True).load("/mnt/Gen2Source/Customer/parquetFiles")

In [0]:
display(df_parquet.show(10))

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
C_CUSTKEY| C_NAME| C_ADDRESS|C_NATIONKEY| C_PHONE|C_ACCTBAL|C_MKTSEGMENT| C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
 12874|Customer#000012874|7MqIGEWlg 4jwK,WZ...| 0|10-592-120-9012| 3521.63| HOUSEHOLD| to the regular d...|
 22282|Customer#000022282|MqdclKuHnUKSTxYAi...| 0|10-537-480-8285| 4131.97| HOUSEHOLD|thely final pinto...|
 1377|Customer#000001377|uUnbwILMLaV8Erfqg...| 0|10-398-963-9520| 8839.15| MACHINERY|s sublate careful...|
 5663|Customer#000005663|Jsi19X,KjM84YxUk0...| 0|10-803-391-3309| 8725.04| AUTOMOBILE|t the quickly eve...|
 16903|Customer#000016903| 79N66cjNpvsMb| 0|10-216-205-5586| -682.56| MACHINERY|ounts. unusual, b...|
 8368|Customer#000008368| GbRjmGOMUQw| 0|10-432-479-9761| 9904.61| FURNITURE|. regular foxes c...|
 17779|Customer#000017779|1Y1JmWXX2iHbxk58k...| 0|10-479-584-6686| -778.25| FURNITURE|ly regular deposi...|
 16513|Customer#000016513|yfjYjofi87FK,Igjq...| 0|10-713-212-7306| -87.27| BUILDING|ideas-- quickly i...|
 14275|Customer#000014275|6,Wh4R2Nv2rM4fDIT...| 0|10-784-143-7613| 3874.94| BUILDING|y bold accounts. ...|
 17316|Customer#000017316|W2KoO5VCF ZZ3mQWQcLP| 0|10-146-891-9644| 4241.91| AUTOMOBILE|was slyly. slyly ...|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
only showing top 10 rows

In [0]:
%sql

ALTER TABLE CustomerDelta SET TBLPROPERTIES ('delta.isolationLevel' = 'Serializable')

In [0]:
df_parquet.write.format("delta").mode("append").save("/mnt/Gen2Source/CustomerDelta")

In [0]:
%sql

SELECT COUNT(*) FROM CustomerDelta

count(1)
1532957


In [0]:
spark.sql("CREATE TABLE CustomerDelta USING DELTA LOCATION '/mnt/Gen2Source/CustomerDelta/'")

Out[27]: DataFrame[]

In [0]:
%sql
SELECT * FROM CustomerDelta LIMIT 10

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
12874,Customer#000012874,"7MqIGEWlg 4jwK,WZIakKqzMa9XYbrZjW",0,10-592-120-9012,3521.63,HOUSEHOLD,to the regular deposits. idly ironic instructions a
22282,Customer#000022282,MqdclKuHnUKSTxYAiPteWcUeT2aLffUGmKMvDv,0,10-537-480-8285,4131.97,HOUSEHOLD,thely final pinto beans might cajole careful
1377,Customer#000001377,uUnbwILMLaV8Erfqgug DvzUxD35ohKtUnal,0,10-398-963-9520,8839.15,MACHINERY,s sublate carefully alongside of the slyly express theodolites. furiously special instructions haggle
5663,Customer#000005663,"Jsi19X,KjM84YxUk04G385",0,10-803-391-3309,8725.04,AUTOMOBILE,t the quickly even decoys! platelets cajole slyly blithely ironic excuses. blithely special pi
16903,Customer#000016903,79N66cjNpvsMb,0,10-216-205-5586,-682.56,MACHINERY,"ounts. unusual, bold accounts boost q"
8368,Customer#000008368,GbRjmGOMUQw,0,10-432-479-9761,9904.61,FURNITURE,. regular foxes cajole along the slyly ironic asymptotes. fluffily ironic theodolites are
17779,Customer#000017779,1Y1JmWXX2iHbxk58k0mf4tzHrFSxihMmtXoKd,0,10-479-584-6686,-778.25,FURNITURE,ly regular deposits sleep slyly. final dependencies according to the fluffily unusual instructions
16513,Customer#000016513,"yfjYjofi87FK,IgjqTZuzHZ",0,10-713-212-7306,-87.27,BUILDING,ideas-- quickly ironic ideas boost alongside of
14275,Customer#000014275,"6,Wh4R2Nv2rM4fDITdJelc 0b6Nvw192oVXa",0,10-784-143-7613,3874.94,BUILDING,y bold accounts. special excuses sleep. accounts sleep blithely. ideas haggle blithely furiously ironic d
17316,Customer#000017316,W2KoO5VCF ZZ3mQWQcLP,0,10-146-891-9644,4241.91,AUTOMOBILE,"was slyly. slyly special deposits run slyly over the ironic, express accounts. silently regular foxes sle"


In [0]:
%sql
UPDATE CustomerDeltaPartition SET C_NATIONKEY = 4

In [0]:
%sql
DELETE FROM CustomerDelta WHERE C_MKTSEGMENT='AUTOMOBILE'

In [0]:
df_parquet.write.format("delta").partitionBy("C_MKTSEGMENT").mode("append").save("/mnt/Gen2/CustomerDeltaPartition")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1869822098999474> in <module> 
 ----> 1 df_parquet . write . format ( "delta" ) . partitionBy ( "C_MKTSEGMENT" ) . mode ( "append" ) . save ( "/mnt/Gen2/CustomerDeltaPartition" ) 

 NameError : name 'df_parquet' is not defined

In [0]:
%sql

ALTER TABLE CustomerDeltaPartition SET TBLPROPERTIES ('delta.isolationLevel' = 'WriteSerializable')

In [0]:
for i in range(1,20):
  df_parquet.where(col("C_MKTSEGMENT")=="MACHINERY").write.format("delta").mode("append").partitionBy("C_MKTSEGMENT").save("/mnt/Gen2Source/CustomerDeltaPartition")

In [0]:
spark.sql("CREATE TABLE CustomerDeltaPartition USING DELTA LOCATION '/mnt/Gen2Source/CustomerDeltaPartition'")

Out[43]: DataFrame[]

In [0]:
%sql
UPDATE CustomerDeltaPartition SET C_NATIONKEY = 4 WHERE C_MKTSEGMENT='MACHINERY'